In [2]:
from bs4 import BeautifulSoup
import pandas as pd 
import numpy as np
import requests
import sqlite3
import datetime
df = pd.DataFrame({"MLS": [],"Street": [], "City":[],"List Price":[],"Bedrooms":[],"Bathrooms":[],"Square Footage":[],"Date":[]})


In [3]:
def scraping(dataframe):
    page=requests.get("http://www.slocountyhomes.com/newlistex.php")
    data=BeautifulSoup(page.text,"html.parser")
    i=0
    date=""
    date="NA"
    MLS=""
    Street=""
    City=""
    List=""
    Bed=""
    Bath=""
    Sq=0
    Date=""
    
    for table in data.find_all('td'):
        if len(table.text.split("November")) != 1:
            date = table.text
        elif table.text.strip() == "":
            if (i != 0):
                # Found missing data
                i += 1
                if (i == 7):
                    i = 0
                    #dataframe.loc[len(dataframe)]=[Bath,Bed,City,date,List,MLS,Sq,Street]
        else:
            if i == 0:
                MLS = table.text.strip()
                i += 1
            elif i == 1:
                Street = sqlize_string(table.text.strip())
                i += 1
            elif i == 2:
                City = sqlize_string(table.text.strip())
                i += 1
            elif i == 3:
                List = int(table.text.strip()[1:].replace(",",""))
                i += 1
            elif i == 4:
                Bed = int(table.text.strip())
                i += 1
            elif i == 5:
                Bath = int(table.text.strip())
                i += 1
            elif i == 6:
                Sq = int(table.text.strip())
                i = 0
                dataframe.loc[len(dataframe)]=[Bath,Bed,City,date,List,MLS,Sq,Street]
                
    dataframe=dataframe.drop_duplicates()
    return dataframe

In [4]:
def sqlize_string(string):
    return "'" + string + "'"

Lot size most important in prediction 


In [5]:
#for td in data.find_all('td'):
#    print(td.text.lstrip().rstrip())

In [6]:
df=scraping(df)

In [7]:
months = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
month_map = {key:int(val) for key,val in zip(months, range(1,13))}
month_map

In [9]:
def insert_houses(df):
    conn = sqlite3.connect('slo_housing.db')
    print("Opened database successfully");
    stmt = "INSERT INTO HOUSES (MLS_ID,BED,BATH,CITY, ADDRESS, LIST_PRICE, SQ_FOOTAGE, PRICE_PER_SQFT, LIST_DATE) \
          VALUES ({}, {}, {}, {}, {}, {}, {}, {}, {})"
    for idx, row in df.iterrows():
        try:
            conn.execute(stmt.format(row['MLS'], row['Bedrooms'], row['Bathrooms'], row['City'],
                                 row['Street'], row['List Price'], row['Square Footage'],
                                 int(row['List Price']) / int(row['Square Footage']), 
                                 datetime.date(2016, month_map[row['Date'].split()[0]], int(row['Date'].split()[1]))
                                ));
        except:
            print(error)
    conn.commit()
    print("Records created successfully");
    conn.close()

In [10]:
insert_houses(df)

In [14]:
conn = sqlite3.connect('slo_housing.db')
check = conn.execute("SELECT * FROM HOUSES")
results = check.fetchall()
conn.close()

In [15]:
df = pd.DataFrame(check.fetchall())

def get_query(query):
    var_names = ['MLS Key','Num Bedrooms','Num Bathrooms', 'City', 'Address', 'Price',
                'Sqr Footage', 'Unknown1', 'Unknown2', 'Unknown3', 'Unknown4', 'Year Sold']
    conn = sqlite3.connect('slo_housing.db')
    df = pd.DataFrame(conn.execute(query).fetchall())
    del df[0]
    df.columns = var_names
    return df
    

In [10]:
get_query('SELECT * FROM HOUSES').head()